In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra

import wandb

from data.dataManager import DataManager
from model.modelCreator import ModelCreator
from omegaconf import OmegaConf
from scripts.run import setup_model, load_model_instance

from utils.plots import vae_plots
from utils.rbm_plots import plot_rbm_histogram

[18:44:24.873] INFO   CaloQuVAE                                         Loading configuration.
[18:44:24.876] INFO   CaloQVAE                                          Willkommen!
[18:44:24.876] INFO   CaloQVAE                                          Loading configuration.


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="config")
config=compose(config_name="config.yaml")
wandb.init(tags = [config.data.dataset_name], project=config.wandb.project, entity=config.wandb.entity, config=OmegaConf.to_container(config, resolve=True), mode='offline')

In [3]:
new_model = True
if new_model:
    self = setup_model(config)
    # self.model = self.model.double()  # sets all model parameters to float64
else:
    self = load_model_instance(config.config_path)
    # self.model = self.model.double()


[21:33:33.137] INFO   data.dataManager                                  Keys: ['incident_energies', 'showers']
[21:33:35.518] INFO   data.dataManager                                  dict_keys(['incident_energies', 'showers'])
[21:33:35.521] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f3763d742f0>: 101815 events, 796 batches
[21:33:35.522] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f36bca69a60>: 12728 events, 13 batches
[21:33:35.522] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f393e550dd0>: 12726 events, 13 batches
[21:33:35.522] INFO   model.modelCreator                                ::Creating Model
[21:33:35.753] INFO   dwave.cloud.client.base                           Fetching definitions of all available solvers
[21:33:35.804] INFO   dwave.cloud.client.base                           

cuda:1
encoder._networks.0.seq1.0.conv.weight True
encoder._networks.0.seq1.0.conv.bias True
encoder._networks.0.seq1.1.weight True
encoder._networks.0.seq1.1.bias True
encoder._networks.0.seq1.2.weight True
encoder._networks.0.seq1.3.conv.weight True
encoder._networks.0.seq1.3.conv.bias True
encoder._networks.0.seq1.4.weight True
encoder._networks.0.seq1.4.bias True
encoder._networks.0.seq1.5.weight True
encoder._networks.0.seq2.0.conv.weight True
encoder._networks.0.seq2.0.conv.bias True
encoder._networks.0.seq2.1.weight True
encoder._networks.0.seq2.1.bias True
encoder._networks.0.seq2.2.weight True
encoder._networks.0.seq2.3.conv.weight True
encoder._networks.0.seq2.3.conv.bias True
encoder._networks.0.seq2.4.weight True
encoder._networks.1.seq1.0.conv.weight True
encoder._networks.1.seq1.0.conv.bias True
encoder._networks.1.seq1.1.weight True
encoder._networks.1.seq1.1.bias True
encoder._networks.1.seq1.2.weight True
encoder._networks.1.seq1.3.conv.weight True
encoder._networks.1.

In [10]:
# self.evaluate(self.data_mgr.val_loader, 0)

In [9]:
# rbm_hist = plot_rbm_histogram(self.RBM_energy_post, self.RBM_energy_prior)

In [8]:
# vae_plots(self.incident_energy, self.showers, self.showers_recon)

In [12]:
self.fit(0)

/usr/local/lib/python3.12/dist-packages/torch/_tensor.py:1128: UserWarning:

Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/Scalar.cpp:22.)

[21:44:38.315] INFO   engine.engine                                     Epoch: 0 [0/796 (0%)]	 beta: 1.000, slope: 0.050 	 Batch Loss: 41711.0547
[21:55:59.832] INFO   engine.engine                                     Epoch: 0 [79/796 (10%)]	 beta: 1.000, slope: 0.050 	 Batch Loss: 3001.2837


KeyboardInterrupt: 